In [1]:
import os
# Import des deux fichiers python contenant nos différentes fonctions
import utils
import crud

### Ouverture de la connexion

In [2]:
bdd = "floupics"
user = "postgres"
psw = os.environ.get("pg_psw")

conn = crud.ouvrir_connection(bdd, user, psw)
type(conn)

psycopg2.extensions.connection

### Création des tables & Insertion des données contenues dans les csv

In [3]:
#Movies table
movie_file = 'CSV/movies.csv'
movie_table = """CREATE TABLE IF NOT EXISTS movies (
                    movieId INT NOT NULL PRIMARY KEY,
                    title TEXT,
                    genres TEXT
                )"""
movie_insert = 'INSERT INTO movies (movieId, title, genres) values (%s, %s, %s)'

crud.create_and_insert(conn, movie_table, movie_insert, movie_file)

In [4]:
#Ratings table
ratings_file = 'CSV/ratings.csv'
ratings_table = """CREATE TABLE IF NOT EXISTS ratings (
                    userId INT,
                    movieId INT NOT NULL,
                    rating REAL,
                    timestamp INT,
                    CONSTRAINT fk_movieId
                        FOREIGN KEY(movieId)
                        REFERENCES movies(movieID),
                    PRIMARY KEY(userId, movieId)
                )"""
ratings_insert = 'INSERT INTO ratings (userId, movieId, rating, timestamp) values (%s, %s, %s, %s)'

crud.create_and_insert(conn, ratings_table, ratings_insert, ratings_file)

In [5]:
#Links table
links_file = 'CSV/links.csv'
links_table = """CREATE TABLE IF NOT EXISTS links (
                    movieId INT NOT NULL PRIMARY KEY,
                    imdbId INT,
                    tmdbId TEXT,
                    CONSTRAINT fk_movieId
                        FOREIGN KEY(movieId)
                        REFERENCES movies(movieId)
              )"""
links_insert = 'INSERT INTO links (movieId, imdbId, tmdbId) values (%s, %s, %s)'

crud.create_and_insert(conn, links_table, links_insert, links_file)

In [6]:
#Tags table
tags_file = 'CSV/tags.csv'
tags_table = """CREATE TABLE IF NOT EXISTS tags (
                    userId INT,
                    movieId INT NOT NULL,
                    tag TEXT,
                    timestamp INT,
                    CONSTRAINT fk_movieId
                        FOREIGN KEY(movieId)
                        REFERENCES movies(movieId),
                    PRIMARY KEY(userId, movieId, tag)
              )"""
tags_insert = 'INSERT INTO tags (userId, movieId, tag, timestamp) values (%s, %s, %s, %s)'


crud.create_and_insert(conn, tags_table, tags_insert, tags_file)

### Traitements sur les données (Ajout des colonnes year & title_only)

In [7]:
from pathlib import Path

# Répertoire dataset (small)
repertoire_data = Path('CSV')

# Chemin vers les différents fichiers de données
chemin_movies = repertoire_data / 'movies.csv'

# Récupération des données années & titres
movies_dict = utils.lire_csv_dict(chemin_movies)
year = []
title_only = []

for i in range(len(movies_dict)):
    try:
        m = utils.renvoyer_annee_titre_film(movies_dict[i]['title'])
        year.append(m[0])
        title_only.append(m[1])
    except:
        print(f"Erreur de lecture sur {movies_dict[i]['title']}")

In [8]:
# Mise à jour des données avec les nouvelles colonnes
crud.update_database(conn, title_only, "title_only", "TEXT")
crud.update_database(conn, year, "year", "INT")

Les données ont étés modifiées
Les données ont étés modifiées


In [9]:
#Affichage des 5 première lignes de la table movies modifiée
select_query = '''SELECT * FROM movies ORDER BY movieId ASC LIMIT 5'''
crud.lire_donnees(conn, select_query)

Les données ont été lues avec succès


[(1,
  'Toy Story (1995)',
  'Adventure|Animation|Children|Comedy|Fantasy',
  'Toy Story',
  1995),
 (2, 'Jumanji (1995)', 'Adventure|Children|Fantasy', 'Jumanji', 1995),
 (3, 'Grumpier Old Men (1995)', 'Comedy|Romance', 'Grumpier Old Men', 1995),
 (4,
  'Waiting to Exhale (1995)',
  'Comedy|Drama|Romance',
  'Waiting to Exhale',
  1995),
 (5,
  'Father of the Bride Part II (1995)',
  'Comedy',
  'Father of the Bride Part II',
  1995)]